In [1]:
#initiate dependencies
import pandas as pd
from pathlib import Path

In [2]:
schools_path = Path('Resources/schools_complete.csv')
schools_df = pd.read_csv(schools_path)
schools_df.head()

,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [3]:
students_path = Path('Resources/students_complete.csv')
students_df = pd.read_csv(students_path)
students_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [22]:
#merge school and student data into one dataframe
district_complete_df = pd.merge(schools_df, students_df, on='school_name', how='left')
district_complete_df.head()

,School ID,school_name,type,size,budget,Student ID,student_name,gender,grade,reading_score,math_score
0,0,Huang High School,District,2917,1910635,0,Paul Bradley,M,9th,66,79
1,0,Huang High School,District,2917,1910635,1,Victor Smith,M,12th,94,61
2,0,Huang High School,District,2917,1910635,2,Kevin Rodriguez,M,12th,90,60
3,0,Huang High School,District,2917,1910635,3,Dr. Richard Scott,M,12th,67,58
4,0,Huang High School,District,2917,1910635,4,Bonnie Ray,F,9th,97,84


District Summary

In [23]:
#Total number of unique schools
school_count = district_complete_df['school_name'].nunique() #schools_df['school_name'].count()
school_count

15

In [24]:
#Total students
student_count = district_complete_df['student_name'].count()
student_count

39170

In [25]:
#Total budget
total_budget = schools_df['budget'].sum()
total_budget

24649428

In [26]:
#Average math score
avg_math = district_complete_df['math_score'].mean()
avg_math

78.98537145774827

In [27]:
#Average reading score
avg_reading = district_complete_df['reading_score'].mean()
avg_reading

81.87784018381414

In [28]:
#% passing math (the percentage of students who passed math)
passing_math = district_complete_df[district_complete_df['math_score'] >= 70].count()['student_name']
pct_passing_math = passing_math / student_count * 100
pct_passing_math

74.9808526933878

In [29]:
#% passing reading (the percentage of students who passed reading)
passing_reading = district_complete_df[district_complete_df['reading_score'] >= 70].count()['student_name']
pct_passing_reading = passing_reading / student_count * 100
pct_passing_reading

85.80546336482001

In [30]:
#% overall passing (the percentage of students who passed math AND reading)
passing_overall = district_complete_df[(district_complete_df['reading_score'] >= 70) & (district_complete_df['math_score'] >= 70)].count()['student_name']
pct_passing_overall = passing_overall / student_count * 100
pct_passing_overall

65.17232575950983

In [31]:
#create a high-level snapshot of the district's key metrics in a DataFrame
district_summary = pd.DataFrame([{
    'Total number of unique schools' : school_count,
    'Total students' : student_count,
    'Total budget' : total_budget,
    'Average math score' : avg_math,
    'Average reading score' : avg_reading,
    '% passing math' : pct_passing_math, # (the percentage of students who passed math)
    '% passing reading' : pct_passing_reading, # (the percentage of students who passed reading)
    '% overall passing' : pct_passing_overall # (the percentage of students who passed math AND reading)
}])

# Format the summary dataframe
district_summary['Total students'] = district_summary['Total students'].map('{:,}'.format)
district_summary['Total budget'] = district_summary['Total budget'].map('${:,.2f}'.format)

district_summary

,Total number of unique schools,Total students,Total budget,Average math score,Average reading score,% passing math,% passing reading,% overall passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


School Summary

In [32]:
# Use the code provided to select all of the school types
school_types = district_complete_df['type'].unique()
school_types

array(['District', 'Charter'], dtype=object)

In [19]:
# Calculate the total student count per school
per_school_counts = district_complete_df['school_name'].value_counts()
per_school_counts

school_name
Bailey High School       4976
Johnson High School      4761
Hernandez High School    4635
Rodriguez High School    3999
Figueroa High School     2949
Huang High School        2917
Ford High School         2739
Wilson High School       2283
Cabrera High School      1858
Wright High School       1800
Shelton High School      1761
Thomas High School       1635
Griffin High School      1468
Pena High School          962
Holden High School        427
Name: count, dtype: int64

In [21]:
# Calculate the total school budget and per capita spending per school
per_school_budget = district_complete_df['school_name','budget'].
per_school_budget
#per_school_capita = 

KeyError: "None of [Index(['Huang High School', 'Huang High School', 'Huang High School',\n       'Huang High School', 'Huang High School', 'Huang High School',\n       'Huang High School', 'Huang High School', 'Huang High School',\n       'Huang High School',\n       ...\n       'Thomas High School', 'Thomas High School', 'Thomas High School',\n       'Thomas High School', 'Thomas High School', 'Thomas High School',\n       'Thomas High School', 'Thomas High School', 'Thomas High School',\n       'Thomas High School'],\n      dtype='object', length=39170)] are in the [index]"

In [ ]:
# Calculate the average test scores per school
per_school_math = 
per_school_reading = 

In [ ]:
# Calculate the number of students per school with math scores of 70 or higher
students_passing_math = 
school_students_passing_math = 

In [ ]:
# Calculate the number of students per school with reading scores of 70 or higher
students_passing_reading = 
school_students_passing_reading = 

In [ ]:
# Use the provided code to calculate the number of students per school that passed both math and reading with scores of 70 or higher
students_passing_math_and_reading = school_data_complete[
    (school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70)
]
school_students_passing_math_and_reading = students_passing_math_and_reading.groupby(["school_name"]).size()

In [ ]:
# Use the provided code to calculate the passing rates
per_school_passing_math = school_students_passing_math / per_school_counts * 100
per_school_passing_reading = school_students_passing_reading / per_school_counts * 100
overall_passing_rate = school_students_passing_math_and_reading / per_school_counts * 100

In [ ]:
# Create a DataFrame called `per_school_summary` with columns for the calculations above.
per_school_summary = 

# Formatting
per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)

# Display the DataFrame
per_school_summary

## Highest-Performing Schools (by % Overall Passing)

In [ ]:
# Sort the schools by `% Overall Passing` in descending order and display the top 5 rows.
top_schools = 
top_schools.head(5)

## Bottom Performing Schools (By % Overall Passing)

In [ ]:
# Sort the schools by `% Overall Passing` in ascending order and display the top 5 rows.
bottom_schools = 
bottom_schools.head(5)

## Math Scores by Grade

In [ ]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by `school_name` and take the mean of the `math_score` column for each.
ninth_grade_math_scores = 
tenth_grader_math_scores = 
eleventh_grader_math_scores = 
twelfth_grader_math_scores = 

# Combine each of the scores above into single DataFrame called `math_scores_by_grade`
math_scores_by_grade = 

# Minor data wrangling
math_scores_by_grade.index.name = None

# Display the DataFrame
math_scores_by_grade

## Reading Score by Grade 

In [ ]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by `school_name` and take the mean of the the `reading_score` column for each.
ninth_grade_reading_scores = 
tenth_grader_reading_scores = 
eleventh_grader_reading_scores = 
twelfth_grader_reading_scores = 

# Combine each of the scores above into single DataFrame called `reading_scores_by_grade`
reading_scores_by_grade = 

# Minor data wrangling
reading_scores_by_grade = reading_scores_by_grade[["9th", "10th", "11th", "12th"]]
reading_scores_by_grade.index.name = None

# Display the DataFrame
reading_scores_by_grade

## Scores by School Spending

In [ ]:
# Establish the bins 
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

In [ ]:
# Create a copy of the school summary since it has the "Per Student Budget" 
school_spending_df = per_school_summary.copy()

In [ ]:
# Use `pd.cut` to categorize spending based on the bins.
school_spending_df["Spending Ranges (Per Student)"] = 
school_spending_df

In [ ]:
#  Calculate averages for the desired columns. 
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Math Score"].mean()
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Math"].mean()
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Reading"].mean()
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Overall Passing"].mean()

In [ ]:
# Assemble into DataFrame
spending_summary = 

# Display results
spending_summary

## Scores by School Size

In [ ]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [ ]:
# Categorize the spending based on the bins
# Use `pd.cut` on the "Total Students" column of the `per_school_summary` DataFrame.

per_school_summary["School Size"] = 
per_school_summary

In [ ]:
# Calculate averages for the desired columns. 
size_math_scores = per_school_summary.groupby(["School Size"])["Average Math Score"].mean()
size_reading_scores = per_school_summary.groupby(["School Size"])["Average Reading Score"].mean()
size_passing_math = per_school_summary.groupby(["School Size"])["% Passing Math"].mean()
size_passing_reading = per_school_summary.groupby(["School Size"])["% Passing Reading"].mean()
size_overall_passing = per_school_summary.groupby(["School Size"])["% Overall Passing"].mean()

In [ ]:
# Create a DataFrame called `size_summary` that breaks down school performance based on school size (small, medium, or large).
# Use the scores above to create a new DataFrame called `size_summary`
size_summary = 

# Display results
size_summary

## Scores by School Type

In [ ]:
# Group the per_school_summary DataFrame by "School Type" and average the results.
average_math_score_by_type = per_school_summary.groupby(["School Type"])["Average Math Score"].mean()
average_reading_score_by_type = per_school_summary.groupby(["School Type"])["Average Reading Score"].mean()
average_percent_passing_math_by_type = per_school_summary.groupby(["School Type"])["% Passing Math"].mean()
average_percent_passing_reading_by_type = per_school_summary.groupby(["School Type"])["% Passing Reading"].mean()
average_percent_overall_passing_by_type = per_school_summary.groupby(["School Type"])["% Overall Passing"].mean()

In [ ]:
# Assemble the new data by type into a DataFrame called `type_summary`
type_summary = 

# Display results
type_summary